## Load Data

### Data Class

In [121]:
import random

class Sentiment: # enum class
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'
    POSITIVE = 'POSITIVE'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: # Score of 4 or 5
            return Sentiment.POSITIVE
        
class ReviewContainer: 
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
        
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
       
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))

        positive_shrunk = positive[:len(negative)]  # Shrink positive review
        
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)  # Important to be random for some algorithmic models
        
#         print(negative[0].text)
#         print("Negative len: " + str(len(negative)))
#         print("Positive len: " + str(len(positive_shrunk)))
        


In [84]:
import json
file_name = "./data/sentiment/Books_small_10000.json"  # Location of the file

reviews = []

with open(file_name) as f: 
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
# reviews[5][1]
print(reviews[5].text)
print(reviews[5].score)
print(reviews[5].sentiment)

I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's.
5.0
POSITIVE


## Prep Data

In [123]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

### Container keeps things neat in the real world scenario
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

cont.evenly_distribute()

To be honest, I didn't quite get very far into this book.  I wanted to learn more about the French, whereas the author seemed caught up in various sexual intrigues.  She seemed more interested in herself than delving into the lives of those who surrounded her.
Negative len: 436
Positive len: 436


In [148]:
train_container.evenly_distribute()

train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

# print(train_y.count(Sentiment.POSITIVE))  # Should be 436
# print(train_y.count(Sentiment.NEGATIVE))  # Should be 436

### Bag of words vectorization

In [158]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# This book is great!
# This book was so bad

# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()

train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)


print(train_x[0])
# print(train_x_vectors[0])
print(train_x_vectors[0].toarray())

This writer has managed to write a compelling True story about thelife of a police force and their humanity in performing their duties!His presentation  was quite good and his written word left meanticipating the outcome.
[[0. 0. 0. ... 0. 0. 0.]]


## Classification

### Linear SVM

In [159]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

test_x[0]
clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Decision Tree

In [160]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Naive Bayes

In [161]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(), train_y)

clf_gnb.predict(test_x_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

### Logistic Regression

In [162]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

## Evaluation

In [168]:
## Mean Accuracy
print("SVM " + str(clf_svm.score(test_x_vectors, test_y)))
print("dec " + str(clf_dec.score(test_x_vectors, test_y)))
print("gnb " + str(clf_gnb.score(test_x_vectors.toarray(), test_y)))
print("log " + str(clf_log.score(test_x_vectors, test_y)))

SVM 0.8076923076923077
dec 0.6442307692307693
gnb 0.6610576923076923
log 0.8052884615384616


In [164]:
# F1 Scores
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
# f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])

/home/antmaker/anaconda3/envs/datascience/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


array([0.80582524, 0.        , 0.80952381])

In [165]:
test_set = ['I thoroughly enjoyed this, 5 stars', 'bad book do not buy', 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

In [166]:
test_y.count(Sentiment.POSITIVE)
test_y.count(Sentiment.NEGATIVE)

208

### Tuning out model (with Grid Search)

In [171]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv = 5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [174]:
print(clf.score(test_x_vectors, test_y))

0.8197115384615384


### Saving Model

In [178]:
import pickle 

with open('./models/sentiment_classifer_anthonyLee.pkl', 'wb') as f:
    pickle.dump(clf, f)

### Load Model

In [180]:
with open ('./models/sentiment_classifer_anthonyLee.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [181]:
test_x[0]
loaded_clf.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')